### Creating public world info vectordb

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
from langchain_core.embeddings import Embeddings
from uuid import uuid4
import chromadb

embeddings = CohereEmbeddings(cohere_api_key="MKDtmSMQq4NJ4DgKcY9Ql4QwF6XWbgIvWRn8FpUC", model="embed-english-v3.0", user_agent='langchain')

with open(f'er_text_documents/dialogue.txt', errors='ignore') as f:
    public_world_info = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap  = 500,
    length_function = len,
    separators = ['\n\n\n', '\n\n', '\n', '.', ',', ' '],
    is_separator_regex=False,
)
documents = text_splitter.create_documents([public_world_info])
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="vectordbs\public_world_info",
)
vector_store.add_documents(documents=documents, ids=uuids)

c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


['dc86acb9-8c73-4240-84eb-4f8b313114f3',
 '022892fd-63bb-4ce2-9123-aba5c42c3c61',
 '47739c2e-cfd9-4b24-a55c-1ca111a21598']

In [3]:
query = "Whats iphone did you recommend?"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    print(f"{res.page_content}")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


User: Hi, I'm interested in purchasing a new smartphone. Can you recommend some options with excellent camera quality?

RAG System: Absolutely! When it comes to smartphones with exceptional camera capabilities, the market offers several impressive choices. One highly regarded option is the Pixel 6 Pro, known for its powerful camera system, including a 50MP main sensor and advanced features like Magic Eraser and Real Tone. Another great choice is the iPhone 13 Pro, which boasts a triple 12MP camera setup with Cinematic mode and ProRes video recording.

User: That's great to know! I'm particularly interested in the iPhone. Can you tell me more about its camera features and any unique capabilities it offers?

RAG System: The iPhone 13 Pro's camera system is truly remarkable. It introduces a new macro photography mode, allowing you to capture stunning close-up shots with incredible detail. The ProRes video recording feature is a game-changer for content creators, offering professional-grad

In [3]:
import shutil
# path to public_vectordb inside your game folder
folder_path = 'vectordbs/public_world_info'
shutil.rmtree(folder_path)